# Data Visualization

In [1]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.neighbors
import matplotlib.pyplot as plt
import ast
import gensim, logging

In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/evaluation.csv')
train_df.describe()

,retweets_count,favorites_count,followers_count,statuses_count,friends_count,verified,timestamp,TweetID
count,353969.000000,353969.000000,3.539690e+05,3.539690e+05,353969.000000,353969.000000,3.539690e+05,3.539690e+05
mean,15.831810,46.655442,2.021548e+04,4.808507e+04,1459.289003,0.030005,1.647004e+12,6.872503e+05
std,241.986723,852.044385,2.598715e+05,1.133854e+05,2502.933271,0.170602,4.846468e+09,4.175793e+05
min,0.000000,0.000000,0.000000e+00,1.000000e+00,0.000000,0.000000,1.301178e+12,3.000000e+00
25%,0.000000,0.000000,1.600000e+02,2.972000e+03,214.000000,0.000000,1.647068e+12,3.194490e+05
50%,1.000000,0.000000,7.260000e+02,1.250100e+04,693.000000,0.000000,1.647292e+12,6.719730e+05
75%,3.000000,1.000000,2.283000e+03,4.352200e+04,1804.000000,0.000000,1.647532e+12,1.049644e+06
max,63674.000000,122591.000000,1.441710e+07,8.183508e+06,237269.000000,1.000000,1.647727e+12,1.434456e+06


In [3]:
train_df.head()

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,timestamp,TweetID
0,rt refarcir macron ans nom prépare,3,0,3682,453535,3628,[],[],0,[],1646978048000,832509
1,populaire,0,0,86,1016,284,[],[],0,[],1647694288000,1388011
2,faut dégager cinglé,3,1,1944,28234,1995,[],[],0,[],1647370048000,63896
3,enseignants mettre prescriptions président rép...,0,0,1,1072,0,[],['https://t.co/rytlted08g'],0,[],1647256282000,979251
4,mafieuse oppressive macron,0,0,13957,25311,10841,[],[],0,[],1647258374000,1040049


In [4]:
train_df_input = train_df.drop(['TweetID', 'timestamp', 'mentions', 'retweets_count', 'text'], axis=1)

train_df_input.urls = train_df_input.urls.apply(ast.literal_eval)
train_df_input.urls = train_df_input.urls.apply(len)

train_df_input.hashtags = train_df_input.hashtags.apply(ast.literal_eval)
train_df_input.hashtags = train_df_input.hashtags.apply(len)

train_df_input.head()

,favorites_count,followers_count,statuses_count,friends_count,urls,verified,hashtags
0,0,3682,453535,3628,0,0,0
1,0,86,1016,284,0,0,0
2,1,1944,28234,1995,0,0,0
3,0,1,1072,0,1,0,0
4,0,13957,25311,10841,0,0,0


In [5]:
test_df_input = test_df.drop(['TweetID', 'timestamp', 'mentions', 'text'], axis=1)

test_df_input.urls = test_df_input.urls.apply(ast.literal_eval)
test_df_input.urls = test_df_input.urls.apply(len)

test_df_input.hashtags = test_df_input.hashtags.apply(ast.literal_eval)
test_df_input.hashtags = test_df_input.hashtags.apply(len)

# k-fold Cross Validation

In [6]:
def kfold_cv(X: pd.DataFrame,
             y: pd.DataFrame,
             n_splits: int = 10,
             method: str = 'knn',
             n_neighbors: int = 1):
    kf = sklearn.model_selection.KFold(n_splits=n_splits, shuffle=True, random_state=123456)
    all_splits = [i for i in kf.split(X)]

    train_mse = []
    val_mse = []

    model = None
    if method == 'knn':
        model = sklearn.neighbors.KNeighborsRegressor(n_neighbors=n_neighbors)

    for k in range(n_splits):
        print('Training split', k)
        train_indexes, val_indexes = all_splits[k]

        train_X = X.iloc[train_indexes].values
        mean = train_X.mean(0)
        std = train_X.std(0)
        train_X = (train_X - mean) / std
        train_y = y.iloc[train_indexes].values

        val_X = X.iloc[val_indexes].values
        val_X = (val_X - mean) / std
        val_y = y.iloc[val_indexes].values

        model.fit(train_X, train_y)

        train_predictions = model.predict(train_X)
        val_predictions = model.predict(val_X)

        train_mse.append(np.abs(train_predictions - train_y).mean())
        val_mse.append(np.abs(val_predictions - val_y).mean())

    return (sum(train_mse) / len(train_mse), sum(val_mse) / len(val_mse))


In [7]:
# for n in range(7, 17, 2):
#     print(n, 'neighbors...')
#     print(kfold_cv(train_df_input, train_df['retweets_count'], n_splits=5, n_neighbors=n))

# Output of above loop
knn_mse = { 1: (0.0022840985090, 8.82755207099830),
            3: (5.2445855803790, 7.50757052066827),
            5: (5.9034562077925, 7.29391667684989),
            7: (6.2303256518832, 7.21462506305364),
            9: (6.4206208103262, 7.22025110296241),
            11: (6.5645450982183, 7.21998145087708),
            13: (6.6775331234898, 7.24705572946240),
            15: (6.7655345805517, 7.28415542343034)}

# Submit simple models

In [8]:
make_submission = False

if make_submission:
    mean = train_df_input.values.mean(0)
    std = train_df_input.values.std(0)

    train_X = (train_df_input.values - mean) /  std
    test_X = (test_df_input.values - mean) / std

    test_ids = test_df[['TweetID']]
    model = sklearn.neighbors.KNeighborsRegressor(n_neighbors=7)
    model.fit(train_X, train_df['retweets_count'])
    test_predictions = model.predict(test_X)

    submission_df = pd.DataFrame(data={'retweets_count': test_predictions})
    submission_df = pd.concat([test_ids, submission_df], axis=1)
    submission_df.to_csv('data/submission.csv', index=False)

# NLP

In [11]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class Corpus(object):
    def __init__(self, text_df: pd.DataFrame):
        self.df = text_df

    def __iter__(self):
        for tweet in self.df['text'].to_list():
            yield tweet.split(' ')

tweets = Corpus(train_df)
model = gensim.models.Word2Vec(tweets, min_count=1, vector_size=256, workers=4)
model.save('models/word2vec.model')

2022-11-14 13:03:05,143 : INFO : collecting all words and their counts
2022-11-14 13:03:05,172 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-11-14 13:03:05,204 : INFO : PROGRESS: at sentence #10000, processed 92512 words, keeping 18699 word types
2022-11-14 13:03:05,255 : INFO : PROGRESS: at sentence #20000, processed 184983 words, keeping 27361 word types
2022-11-14 13:03:05,303 : INFO : PROGRESS: at sentence #30000, processed 277832 words, keeping 33853 word types
2022-11-14 13:03:05,342 : INFO : PROGRESS: at sentence #40000, processed 370444 words, keeping 39273 word types
2022-11-14 13:03:05,384 : INFO : PROGRESS: at sentence #50000, processed 462540 words, keeping 43943 word types
2022-11-14 13:03:05,429 : INFO : PROGRESS: at sentence #60000, processed 555094 words, keeping 47971 word types
2022-11-14 13:03:05,488 : INFO : PROGRESS: at sentence #70000, processed 646885 words, keeping 51526 word types
2022-11-14 13:03:05,535 : INFO : PROGRESS: at s